# Project week two

## Importing libraries

In [1]:
import pandas as pd
import re
import operator

## Information about the dataset

The original description of the dataset is: 

*"There are some big plane crashes recently. I want to know more about the crashes. For very first step, I need to collect data from somewhere, then I found http://www.planecrashinfo.com/database.htm You guys can pull new data from planecrashinfo.com by using https://github.com/hocnx/planecrashinfo_scraping"*

And the description of the column headers:

1. date:    Date of accident,  in the format - January 01, 2001
2. time:    Local time, in 24 hr. format unless otherwise specified
3. location: location information
4. Airline/Op:  Airline or operator of the aircraft
5. flight_no:   Flight number assigned by the aircraft operator
6. route:   Complete or partial route flown prior to the accident
7. ac_type:     Aircraft type
8. registration:    ICAO registration of the aircraft
9. cn_ln:   Construction or serial number / Line or fuselage number
10. aboard:  Total aboard (passengers / crew)
11. fatalities:  Total fatalities aboard (passengers / crew)
12. ground:  Total killed on the ground
13. summary:     Brief description of the accident and cause if known

## Reading dataset file

We have the data stored on a csv file, so we use pandas utilities in order to read the file.

In [2]:
plane_crashes_data = pd.read_csv('plane_crash_info.csv')

A briefly view of the data

In [3]:
plane_crashes_data.tail(50)

,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
5733,"March 19, 2016",0341,"Rostov-on-Don, Russia",Flydubai,981,Dubai - Rostov,Boeing 737-8KN (WL),A6-FDN,40241/3517,62 (passengers:55 crew:7),62 (passengers:55 crew:7),0,"At 2237, at Rostov-on-Don, the crew aborted a ..."
5734,"March 29, 2016",1140,"Magdelen Islands, Quebec, Canada",Marquise Aviation Corp.,?,Montreal - Iles-de-la-Magdelen,Mitsubishi MU-2B 60 Marquise,N246W,1552S-A,7 (passengers:5 crew:2),7 (passengers:5 crew:2),0,The plane crashed while on approach to Iles-de...
5735,"April 04, 2016",1530,"Near Pigeon Forge, Tennessee",Smoky Mountain Helicopters,?,Sightseeing,Bell 206L,N16760,45081,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,The helicopter crashed in a wooded hillside wh...
5736,"April 13, 2016",1420,"Kiunga, Papua New Guinea",Sunbird Aviation,?,Oksapmin - Kiungaraining,Britten-Norman Islander,P2-SBC,3010,12 (passengers:11 crew:1),12 (passengers:11 crew:1),0,"While on approach, the plane stalled and crase..."
5737,"April 29, 2016",1150,"Sorta, Norway",CHC Helikopter Service,?,Gullfflex B - Bergen,Eurocopter EC225LP Super Puma M2+,LN-OJF,2721,13 (passengers:11 crew:2),13 (passengers:11 crew:2),0,The helicopter crashed on the rocky shores of ...
5738,"May 19, 2016",0249,175 miles off the Egyptian coast,EgyptAir,804,Paris - Cairo,Airbus A320-232,SU-GCC,2088,66 (passengers:56 crew:10),66 (passengers:56 crew:10),0,"The plane was en route from Paris to Cairo, fl..."
5739,"July 22, 2016",?,"Bay of Bengal, Indian Ocean",Indian Air Force,?,Chennai (Tambram AFB) - Port Blair,Antonov 32,K2743,0809,29 (passengers:23 crew:6),29 (passengers:23 crew:6),0,The aircraft was taking part in fire fighting ...
5740,"July 30, 2016",0742,"Near Lockhart, Texas",Heart of Texas Hot Air Balloon Rides,?,Sightseeing Fentress Airpark,Kubicek BB85Z Hot Air Balloon,N2469L,?,16 (passengers:15 crew:1),16 (passengers:15 crew:1),?,The hot air balloon struck power lines and cra...
5741,"August 05, 2016",1238,"Dubai, United Arab Emirates",Emirates,521,"Thiruvanthapuram, India - Dubai, UAE",Boeing 777-31H,A6-EMW,32700/434,300 (passengers:282 crew:18),0 (passengers:0 crew:0),1,"After being cleared to land, the crew informed..."
5742,"August 31, 2016",1100,"Russian Mission, Alaksa",Hageland Aviiation,3190,Russian Mission- Marshall,Cessna 208 Grand Caravan,N752RV,208B5088,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,A midair collision occurred between a Cessna 2...


## Checking missing values

The first attempt to check missing values with 'isna()' didn't work, because all the missing values were replace with '?'

In [4]:
nulls_col = plane_crashes_data.isna().sum()
nulls_col[nulls_col > 0]

Series([], dtype: int64)

I had to search all the '?' in the dataframe

In [5]:
missings_data = (plane_crashes_data == '?').sum()
print('Missing values:')
missings_data[missings_data > 0]

Missing values:


time            2108
location           6
operator          21
flight_no       4436
route           1494
ac_type           24
registration     352
cn_ln           1207
ground            52
summary          231
dtype: int64

With this we were able to see that there are not missing values for: [date,aboard,fatalities] but in the case of "aboard" and "fatalities" it doesn't mean that we don't have problems because this columns contain a string (it is a value) where it can contain missing values. 

In [6]:
# So we make a little more complex search in that columns
check_aboard_miss = lambda x: True if len(re.findall(r'[?]',str(x))) > 0 else False
aboard_missings = plane_crashes_data['aboard'].apply(check_aboard_miss)
fatalities_missings = plane_crashes_data['fatalities'].apply(check_aboard_miss)
abo_fatal_missing = pd.DataFrame()
abo_fatal_missing['aboard_m'] = aboard_missings
abo_fatal_missing['fatalities_m'] = fatalities_missings
missing_abo_fatal = (abo_fatal_missing == False).sum()
print('Missing values:')
missing_abo_fatal

Missing values:


aboard_m        5239
fatalities_m    5222
dtype: int64

With this we can see that we have: 
1. 544 complete info registers for aboard
2. 561 complete info for fatalities. 

## Data cleaning

I decided to clean each column using the order of appearance.

### Date

Here we have the date of the crash, but it is format in a single string with the "Month day, year" structure but
I considered that it is more useful to have three columns with each element of the date.

In [7]:
# Search in the date column all the letters to obtain the Month
months = [''.join(re.findall(r'[A-Za-z]',plane_crashes_data['date'][da])) for da in range(len(plane_crashes_data['date']))]

In [8]:
# Search in the date column the group of two digits for the day
days = [''.join(re.findall(r'\b\d{2}\b',plane_crashes_data['date'][da])) for da in range(len(plane_crashes_data['date']))]

In [9]:
# Search in the date column the group of four digits for the day
years = [''.join(re.findall(r'\b\d{4}\b',plane_crashes_data['date'][da])) for da in range(len(plane_crashes_data['date']))]

### Time

In the time column we don't have a homogenus format so we check the formats that exist.

In [10]:
plane_crashes_data['time'].head(10)

0      17:18
1          ?
2      06:30
3          ?
4    c 18:30
5    c 10:30
6    c 01:00
7    c 15:20
8          ?
9    c 01:00
Name: time, dtype: object

There is not information about the 'c' character in the time so, we create a new column with this value.
Note: Investigate the meaning.

In [11]:
check_c_char = lambda x: True if len(re.findall(r'[c]',str(x))) > 0 else False
char_unknow = plane_crashes_data['time'].apply(check_c_char)
char_unknow = char_unknow.replace(True, value='c')
char_unknow = char_unknow.replace(False, value='not c')

Then I decided to split this column into two new columns for time_hour and time_minutes, in order to facilitate further filtering tasks.

In [12]:
check_time_for = lambda x: True if len(re.findall(r'[:]',str(x))) > 0 else False

#If we have a different format than "hh:mm" we reformat and if we have the 'c' character we drop it
def reformat(x):
    if check_c_char(x):
        x = x.replace("c"," ")
    if not check_time_for(x) and x != '?':
        x = x[:2] + ':' + x[2:]
    return x

get_hour_time = lambda x: x.replace(' ','')[:2] if x != '?' else x
get_minutes_time = lambda x: x.replace(' ','')[3:] if x != '?' else x

new_times = plane_crashes_data['time'].apply(reformat)
time_hour = new_times.apply(get_hour_time)
time_minutes = new_times.apply(get_minutes_time)


### Location

With the location column we can see that in the major we have a "region,Country" structure so we try to split the values using commas and identify which one is the country. Also we can identify that when the tragedy happen in the United States they use a "region, State" structure so we also have to identify if the element in the column belongs to the United States, for that I create a csv with the Unite States states.

In [13]:
# Load states.csv
states = pd.read_csv('states.csv')
states = states['state'].to_list()

#### Identify the country.

In [14]:
split_lb = lambda x: x.split(',')
check_if_state = lambda x: True if x in states else False

region_country = pd.DataFrame(plane_crashes_data['location'].apply(split_lb))

def who_is_country(li_country):
    temp = []
    for i in li_country:
        if len(i) > 1:
            if i[0] != ' ':
                if check_if_state(i):
                    temp.append('United States')
                else:
                    temp.append(i)
            else:
                if check_if_state(i[1:]):
                    temp.append('United States')
                else:
                    temp.append(i[1:])
        else:
            return li_country
    return temp[-1]
countries = region_country['location'].apply(who_is_country)

### Operator

We can see that there are a lot of kinds in the operator column.

In [15]:
print(len(set(plane_crashes_data['operator'])))

2821


I tried to get a little more homogenus column incorporating each operator in a same category if the word 'Military' is in the string.

In [16]:
check_is_militar = lambda x: 'Military' if len(re.findall(r'(?:Military|Forces?|Fuerzas?|Army)',x,re.I)) > 0 else x
new_operators = plane_crashes_data['operator'].apply(check_is_militar)
print(len(set(new_operators)))

2553


A category for Government

In [17]:
check_is_gover = lambda x: 'Government' if len(re.findall(r'(?:Government|Gobierno)',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_is_gover)
print(len(set(new_operators)))

2547


Also if the word 'taxi' is in the string.

In [18]:
check_is_taxi = lambda x: 'Taxi' if len(re.findall(r'Taxi',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_is_taxi)
print(len(set(new_operators)))

2175


Also there is a small category of operators with the word 'Helicopters' is in the string.

In [19]:
check_is_helicopter = lambda x: 'Helicopter' if len(re.findall(r'Helicopters?',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_is_helicopter)
print(len(set(new_operators)))

2142


Also the major of the comercial operators contain the word 'Airlines' or 'Airways' in the string.

In [20]:
check_airline_airways = lambda x: 'Airlines-Airways operator' if len(re.findall(r'(?:Airlines?|Airways?|Air lines?|Aerolineas?|Lineas?)',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_airline_airways)
print(len(set(new_operators)))

1510


A small category of 'Services' operators,it could be the same than a commercial 'Airlines-Airways', but we keep with a different category.

In [21]:
check_service = lambda x: 'Services' if len(re.findall(r'(?:Services?|Servicios?)',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_service)
print(len(set(new_operators)))

1373


A small category of 'Transport' operators,it could be the same than a 'Services' category, but we keep with a different category.

In [22]:
check_transport = lambda x: 'Transport' if len(re.findall(r'Transports?',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_transport)
print(len(set(new_operators)))

1281


A category for 'Companies' operators.

In [23]:
check_transport = lambda x: 'Company' if len(re.findall(r'(?:Companys?|Compañias?|\sInc.)',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_transport)
print(len(set(new_operators)))

1237


A category for 'Air Company' operators.

In [24]:
check_transport = lambda x: 'Air' if len(re.findall(r'Air?',x,re.I)) > 0 else x
new_operators = new_operators.apply(check_transport)
print(len(set(new_operators)))

753


Finally if the operator is not in one of the categories defined by me, I put on a 'No category' category

In [25]:
new_operators_list = ['Military','Government','Taxi','Helicopter','Airlines-Airways operator','Services','Transport','Company','Air']

In [26]:
check_has_category = lambda x: x if x in new_operators_list else 'No category'
category_operator = new_operators.apply(check_has_category)

### Route

In [27]:
print(len(set(plane_crashes_data['route'])))

3875


We can see that there are a lot of different routes, but in this case there are a lot of variations "Origin-destiny","Origin-Scale-Destiny" and also the names has '-' character what makes so complicated try to clean this row, so I didn't change.

### Aircraft type.

In [28]:
words_ac_type = [i.split(' ') for i in plane_crashes_data['ac_type'].tolist()]
words_ac_type = [i for x in words_ac_type for i in x]

In [29]:
freq_words_ac = {k:words_ac_type.count(k) for k in set(words_ac_type)}

In [30]:
sorted_tups = sorted(freq_words_ac.items(), key=operator.itemgetter(1))

For Aircraft type I just identifided the categories: ['Boeing','Douglas','Lockheed','Cessna','Antonov','Ilyushin','Piper','Fokker'], in order to get a new homogenus column.

In [31]:
check_boeing = lambda x: 'Boeing' if len(re.findall(r'boeing',x,re.I)) > 0 else x
check_lockheed = lambda x: 'Lockheed' if len(re.findall(r'lockheed',x,re.I)) > 0 else x
check_douglas = lambda x: 'Douglas' if len(re.findall(r'boeing',x,re.I)) > 0 else x
check_cessna = lambda x: 'Cessna' if len(re.findall(r'lockheed',x,re.I)) > 0 else x
check_antonov = lambda x: 'Antonov' if len(re.findall(r'boeing',x,re.I)) > 0 else x
check_ilyushin = lambda x: 'Ilyushin' if len(re.findall(r'lockheed',x,re.I)) > 0 else x
check_piper = lambda x: 'Piper' if len(re.findall(r'boeing',x,re.I)) > 0 else x
check_fokker = lambda x: 'Fokker' if len(re.findall(r'lockheed',x,re.I)) > 0 else x

new_ac_type = plane_crashes_data['ac_type'].apply(check_boeing)
new_ac_type = new_ac_type.apply(check_lockheed)
new_ac_type = new_ac_type.apply(check_douglas)
new_ac_type = new_ac_type.apply(check_cessna)
new_ac_type = new_ac_type.apply(check_antonov)
new_ac_type = new_ac_type.apply(check_ilyushin)
new_ac_type = new_ac_type.apply(check_piper)
new_ac_type = new_ac_type.apply(check_fokker)

new_ac_list = ['Boeing','Douglas','Lockheed','Cessna','Antonov','Ilyushin','Piper','Fokker']
check_ac_has_category = lambda x: x if x in new_ac_list else 'No category'
category_ac_type = new_ac_type.apply(check_ac_has_category)

### Flight number, Registration and Construction numbers.

For this three columns we can obtain few information in our analysis because these are numbers derivate from the country or unique numbers for identification of the flights and planes. So I decided to drop it.

In [32]:
columns_remove = ['flight_no','registration','cn_ln']
plane_crashes_data = plane_crashes_data.drop(columns_remove,axis=1)
plane_crashes_data.head()

,date,time,location,operator,route,ac_type,aboard,fatalities,ground,summary
0,"September 17, 1908",17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,Air show,Wright Byplane,1 (passengers:0 crew:1),1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",06:30,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,Curtiss seaplane,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",c 18:30,Over the North Sea,Military - German Navy,?,Zeppelin L-1 (airship),20 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...


### Aboard and Fatalities

For this two we have identifided the missing values but the major of times that missing values refers to the information about how many of the tripulation were passenger or crew. So I extracted the value of total people aboard and the fatalities, that are the number out of the parenthesis.

In [33]:
extract_out_par = lambda x: x[:x.index('(')].replace(' ','') if x != '?' else -1

In [34]:
extract_out_par('19   (passengers:? ')

'19'

In [35]:
aboard_total = plane_crashes_data['aboard'].apply(extract_out_par)
fatalities_total = plane_crashes_data['fatalities'].apply(extract_out_par)

### Ground

In this column we have just a few amount of missing values (52) so I decided to fill them with zero values.

In [36]:
(plane_crashes_data['ground'] == '?').sum()

52

In [37]:
change_miss = lambda x: 0 if x == '?' else x
ground_fill = plane_crashes_data['ground'].apply(change_miss)
(ground_fill == '?').sum()

0

### Summary

# Results

After this modifications we add the new columns to the original dataframe and then we order all the columns.

In [38]:
columns_remove = ['date','time','location','aboard','fatalities','ground']
plane_crashes_data = plane_crashes_data.drop(columns_remove,axis=1)
plane_crashes_data.head()

,operator,route,ac_type,summary
0,Military - U.S. Army,Demonstration,Wright Flyer III,"During a demonstration flight, a U.S. Army fly..."
1,?,Air show,Wright Byplane,Eugene Lefebvre was the first pilot to ever be...
2,Military - U.S. Navy,Test flight,Dirigible,First U.S. dirigible Akron exploded just offsh...
3,Private,?,Curtiss seaplane,The first fatal airplane accident in Canada oc...
4,Military - German Navy,?,Zeppelin L-1 (airship),The airship flew into a thunderstorm and encou...


In [39]:
plane_crashes_data['month'] = months
plane_crashes_data['day'] = days
plane_crashes_data['year'] = years

plane_crashes_data['time_hour'] = time_hour
plane_crashes_data['time_minutes'] = time_minutes

plane_crashes_data['char_unknow'] = char_unknow

plane_crashes_data['country'] = countries

plane_crashes_data['category_operator'] = category_operator
plane_crashes_data['category_ac_type'] = category_ac_type
plane_crashes_data['aboard_people'] = aboard_total
plane_crashes_data['fatalities_total'] = fatalities_total

plane_crashes_data['ground_deads'] = ground_fill


In [40]:
# Ordenamos
column_order = ['month','day','year','char_unknow','time_hour','time_minutes','operator','category_operator','ac_type','category_ac_type','country','route','aboard_people','fatalities_total','ground_deads','summary']
plane_crash_clean = plane_crashes_data[column_order]
plane_crash_clean.head(10)

,month,day,year,char_unknow,time_hour,time_minutes,operator,category_operator,ac_type,category_ac_type,country,route,aboard_people,fatalities_total,ground_deads,summary
0,September,17,1908,not c,17,18,Military - U.S. Army,Military,Wright Flyer III,No category,United States,Demonstration,2,1,0,"During a demonstration flight, a U.S. Army fly..."
1,September,07,1909,not c,?,?,?,No category,Wright Byplane,No category,France,Air show,1,1,0,Eugene Lefebvre was the first pilot to ever be...
2,July,12,1912,not c,06,30,Military - U.S. Navy,Military,Dirigible,No category,United States,Test flight,5,5,0,First U.S. dirigible Akron exploded just offsh...
3,August,06,1913,not c,?,?,Private,No category,Curtiss seaplane,No category,Canada,?,1,1,0,The first fatal airplane accident in Canada oc...
4,September,09,1913,c,18,30,Military - German Navy,Military,Zeppelin L-1 (airship),No category,Over the North Sea,?,20,14,0,The airship flew into a thunderstorm and encou...
5,October,17,1913,c,10,30,Military - German Navy,Military,Zeppelin L-2 (airship),No category,Germany,?,30,30,0,Hydrogen gas which was being vented was sucked...
6,March,05,1915,c,01,00,Military - German Navy,Military,Zeppelin L-8 (airship),No category,Belgium,?,41,21,0,Crashed into trees while attempting to land af...
7,September,03,1915,c,15,20,Military - German Navy,Military,Zeppelin L-10 (airship),No category,Germany,?,19,19,0,"Exploded and burned near Neuwerk Island, when..."
8,July,28,1916,not c,?,?,Military - German Army,Military,Schutte-Lanz S-L-10 (airship),No category,Bulgeria,?,20,20,0,"Crashed near the Black Sea, cause unknown."
9,September,24,1916,c,01,00,Military - German Navy,Military,Zeppelin L-32 (airship),No category,England,?,22,22,0,Shot down by British aircraft crashing in flames.


Finally I decided to change all the missing values (in this case '?') for the 'Unknown' string value

In [41]:
change_to_unknown = lambda x: 'Unknown' if str(x) == '?' else x
for i in plane_crash_clean.columns:
    plane_crash_clean[i] = plane_crash_clean[i].apply(change_to_unknown)

In [42]:
plane_crash_clean.to_csv('data_plane_clean.csv')
plane_crash_clean

,month,day,year,char_unknow,time_hour,time_minutes,operator,category_operator,ac_type,category_ac_type,country,route,aboard_people,fatalities_total,ground_deads,summary
0,September,17,1908,not c,17,18,Military - U.S. Army,Military,Wright Flyer III,No category,United States,Demonstration,2,1,0,"During a demonstration flight, a U.S. Army fly..."
1,September,07,1909,not c,Unknown,Unknown,Unknown,No category,Wright Byplane,No category,France,Air show,1,1,0,Eugene Lefebvre was the first pilot to ever be...
2,July,12,1912,not c,06,30,Military - U.S. Navy,Military,Dirigible,No category,United States,Test flight,5,5,0,First U.S. dirigible Akron exploded just offsh...
3,August,06,1913,not c,Unknown,Unknown,Private,No category,Curtiss seaplane,No category,Canada,Unknown,1,1,0,The first fatal airplane accident in Canada oc...
4,September,09,1913,c,18,30,Military - German Navy,Military,Zeppelin L-1 (airship),No category,Over the North Sea,Unknown,20,14,0,The airship flew into a thunderstorm and encou...
5,October,17,1913,c,10,30,Military - German Navy,Military,Zeppelin L-2 (airship),No category,Germany,Unknown,30,30,0,Hydrogen gas which was being vented was sucked...
6,March,05,1915,c,01,00,Military - German Navy,Military,Zeppelin L-8 (airship),No category,Belgium,Unknown,41,21,0,Crashed into trees while attempting to land af...
7,September,03,1915,c,15,20,Military - German Navy,Military,Zeppelin L-10 (airship),No category,Germany,Unknown,19,19,0,"Exploded and burned near Neuwerk Island, when..."
8,July,28,1916,not c,Unknown,Unknown,Military - German Army,Military,Schutte-Lanz S-L-10 (airship),No category,Bulgeria,Unknown,20,20,0,"Crashed near the Black Sea, cause unknown."
9,September,24,1916,c,01,00,Military - German Navy,Military,Zeppelin L-32 (airship),No category,England,Unknown,22,22,0,Shot down by British aircraft crashing in flames.
